In [1]:
from src.game_model import GameModel

## Declare environment and agents:

In [2]:
"""
Initializes the GameModel with a labeled environment.

Args:
    default_label (list of str): The default labels to assign to each column in the environment.
    additional_labels (list of str): Other possible labels that can be assigned to the environment elements.

    default_agents_labels (list of str)): The default labels to each row of agent features.
    additional_agents_labels (list of list of strings): For each agent feature, specify other possible labels that can be assigned to each.

    environment_dimensions (list of int): Dimensions of the environment matrix. 
    Consider adding an additional dimension to the environment for time if rules refer to previous timesteps.

    agents_dimensions (list of int): Dimensions of the agents matrix.
    Consider to use the first dimension for the number of player and the second dimension for the number of features.

    dimensions_descriptions (str): String explaining each dimension of the environment and of the agents (optional)
    game_name (str): String with the name of the game (optional)
"""

gm = GameModel( 
agents_number=2, default_agent_features=['not starter', 'X'], additional_agent_features=[['starter'], ['O']], 
agent_features_descriptions="2 players with feature 1 indicating who is starting, and feature 2 indicating their symbol.",
game_name="tic-tac-toe")
gm.add_action_space("board", dimensions=[3, 3], default_labels=['free'], additional_labels=[['X', 'O']], dimensions_descriptions="3x3 board.")

print(gm)
print()
print(gm.printable_rules)


        You can add spaces on which to perform actions with the function add_action_space(dimensions, default_labels, additional_labels, dimensions_description).
        Examples of that are the environment space (or board) and the agent space, which are by default already there.

        You can use the method gm.action_is_violation_if(rule, rule_description) to express rules for the game.

        Use help(gm.action_is_violation_if) for help on how to define rules.
        The rule: "Nothing is allowed if the game is ended." is defined by default, to delete it use gm.delete_rule(1).
        Use gm.print_rules() to see all the rules that have been setted.
        
        Use gm.set_endgame(callable_function(game)) to define based on what dynamics your game should end.
        The callable function should have one only input parameter, which represents the game itself, 
        which can be used to refer to anything inside its model, this include:
                - action_spaces = th

In [3]:
# Let's disable actions on the agent feature space.
gm.disable_actions(on="agent")
print(gm.printable_action_spaces)

Action Spaces:

1. agent (actions disabled): 
(2, 2), Number of elements: 4 
Available labels: [['not starter', 'starter'], ['X', 'O']])

Dimensions descriptions: 2 players with feature 1 indicating who is starting, and feature 2 indicating their symbol.

agent: 
[['not starter' 'X']
 ['not starter' 'X']]

2. board (actions allowed): 
(3, 3), Number of elements: 9 
Available labels: [['free', 'X', 'O']])

Dimensions descriptions: 3x3 board.

board: 
[['free' 'free' 'free']
 ['free' 'free' 'free']
 ['free' 'free' 'free']]




## Initialize parameters

In [4]:
gm.agents[1, 1] = 'O'
gm.agents[1, 0] = 'starter'
print(gm.agents)

[['not starter' 'X']
 ['starter' 'O']]


## Define the dynamic to end the game

In [5]:
def tic_tac_toe_endgame(game):
    board = game.action_spaces["board"]
    # Check rows for winning condition
    for row in board:
        if row[0] == row[1] == row[2] != 'free':
            return True

    # Check columns for winning condition
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != 'free':
            return True

    # Check diagonals for winning condition
    if board[0][0] == board[1][1] == board[2][2] != 'free':
        return True
    if board[0][2] == board[1][1] == board[2][0] != 'free':
        return True

    # If no winner, return False
    return False

gm.set_endgame(tic_tac_toe_endgame)

## Define the rules

Define rules by specifying a conditional that is true if the rule is broken. 
The conditional needs to be a callable function with arguments who, where, what and game.

Rules contrain a given action space by limiting the possible actions appliable.

Specify the action space that the rule applies to after the callable function (as second argument).
If you don't specify anything, the rules are applied to all the action spaces.

In [6]:
"""Only the player whose starter can play the first turn:
game.agents[who] = the agent who does the action
agent[0] refers to the agent's first feature, which we assigned to the status
"""
gm.action_is_violation_if(lambda who, where, what, game: not game.started and game.agents[who][0] != 'starter', rule_description="This is not the starting player and this is the first turn.")

In [7]:
"""Agents need to alternate actions:
The last action was done by the same player.
"""
gm.action_is_violation_if(lambda who, where, what, game: game.started and who == game.actions[-1]['who'], rule_description="Players cannot play two times consecutively")

In [8]:
"""You can only put a sign if the space is free:
"""
gm.action_is_violation_if(lambda who, where, what, game: where != 'free', "board", rule_description="The space needs to be free to put a sign on it.")

In [9]:
"""Agents can only put their own sign:
agent[1] refers to the agent's symbol.
"""
gm.action_is_violation_if(lambda who, where, what, game: game.agents[who][1] != what, rule_description="Agents can only put their own sign.")

In [10]:
print(gm.printable_rules)

Rules:

general:
1: Nothing is allowed if the game is ended.
2: This is not the starting player and this is the first turn.
3: Players cannot play two times consecutively
4: Agents can only put their own sign.

agent:

board:
1: The space needs to be free to put a sign on it.




## Example Actions:

In [11]:
board = gm.action_spaces["board"]

Action not permitted:

In [12]:
gm.action("board", 0, [0, 0], 'X')
board

Broke general rule 2: This is not the starting player and this is the first turn.
This action is not permitted.



ActionSpace([['free', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

Action that respects the rules:

In [13]:
board.available_labels

[['free', 'X', 'O']]

In [14]:
gm.action("board", 1, [0, 0], 'O')
board

ActionSpace([['O', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

Other actions that do not respect the rules:

In [15]:
gm.action("board", 1, [1, 0], 'O')
board

Broke general rule 3: Players cannot play two times consecutively
This action is not permitted.



ActionSpace([['O', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

In [16]:
gm.action("board", 0, [0, 0], 'X')
board

Broke rule 1: The space needs to be free to put a sign on it.
This action is not permitted.



ActionSpace([['O', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

In [17]:
gm.action("agent", 0, 'O', [0,0])
board

Actions are not allowed in this Action Space.
This action is not permitted.



ActionSpace([['O', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

In [18]:
gm.action("board", 0, [2, 1], 'O')
board

Broke general rule 4: Agents can only put their own sign.
This action is not permitted.



ActionSpace([['O', 'free', 'free'],
             ['free', 'free', 'free'],
             ['free', 'free', 'free']], dtype='<U4')

## Example of game:

It's suggested to make a wrapper definition for actions in the game:

In [19]:
def put_sign(player, coordinates):
    if player == 0:
        sign = 'X'
    elif player == 1:
        sign = 'O'
    else:
        raise ValueError("Player variable has to be 0 or 1")

    performed = gm.action("board", player, coordinates, sign)

    if performed:
        print(gm.action_spaces["board"])

In [20]:
put_sign(0, [1,1])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['free' 'free' 'free']]


In [21]:
put_sign(0, [0,1])

Broke general rule 3: Players cannot play two times consecutively
This action is not permitted.



In [22]:
put_sign(1, [2,2])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['free' 'free' 'O']]


In [23]:
put_sign(0, [2,0])

[['O' 'free' 'free']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [24]:
put_sign(1, [0,2])

[['O' 'free' 'O']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [25]:
gm.ended

False

In [26]:
previous_board = gm.theoretical_unapply_actions(3)["board"]

In [27]:
previous_board

ActionSpace([['O', 'free', 'free'],
             ['free', 'X', 'free'],
             ['free', 'free', 'O']], dtype='<U4')

In [28]:
put_sign(0, [0,0])

Broke rule 1: The space needs to be free to put a sign on it.
This action is not permitted.



In [29]:
put_sign(0, [0,1])

[['O' 'X' 'O']
 ['free' 'X' 'free']
 ['X' 'free' 'O']]


In [30]:
put_sign(1, [1,2])

[['O' 'X' 'O']
 ['free' 'X' 'O']
 ['X' 'free' 'O']]


In [31]:
gm.ended

True

In [32]:
put_sign(0, [2,1])

Broke general rule 1: Nothing is allowed if the game is ended.
This action is not permitted.

